# Clase 5 - XlsxWriter, estructura de un proyecto y ejemplos prácticos 💹📋

## XlsxWriter

Se instala con `pip install XlsxWriter` o `conda install XlsxWriter`

Es una libreria de python que sirve para crear hojas de calculo de formato "Excel" de forma automatizada y con una capacidad de configuración enorme. Con XlsxWriter vamos a poder crear archivos excel de múltiples hojas a partir de muchos dataframes. Además nos va a dar la posibilidad de crear archivos excel usando casi todo lo que excel nos ofrece. Vamos a poder escribir formulas de excel, dar formato a las celdas, automatizar la creación de gráficos de excel o hasta escribir macros programáticamente.
Muchas veces por más automatizados y complejos que sean nuestros procesos es necesario compartirlos en forma de hojas de cálculo simplemente por la facilidad y familiaridad que tiene esa herramienta. Cuándo queremos exportar proyectos complejos a un excel es usual que un excel de una sola hoja con una tabla plana no sea suficiente satisfacer el objetivo del proyecto, para eso podemos usar esta libreria para sacarle más provecho al formato de excel.

Veamos ejemplos:

In [4]:
import xlsxwriter

workbook = xlsxwriter.Workbook('hello.xlsx')
worksheet = workbook.add_worksheet()

worksheet.write('A1', 'Hello world')

workbook.close()

En este ejemplo ven como crea un nuevo archivo de excel y explícitamente en escribe algo en la celda 'A1'.
Quiero que noten como debe usar el método `.close()` para dejar de trbajar con el archivo. Es super importante no olvidarse de cerrar los archivos porque al igual que cuando uno usa un excel, ninguna otra persona ni proceso puede modificarlo mientras esté abierto. No cerrarlo va a causar que no sea seguro que los procesos que quisimos plasmar en el excel se guarden y tambien puede generar que su programa no pueda terminar correctamente o que directamente se quede abierto imposibilitando usar el archivo.

Por esa razón, para no olvidarse de cerrar el archivo, recomiendo mucho esta otra forma de escribir el mismo código:

In [ ]:
with xlsxwriter.Workbook('hello.xlsx') as workbook:
    worksheet = workbook.add_worksheet()

    worksheet.write('A1', 'Hello world')


De esa forma abrimos o creamos el documento con `with` y una vez que salimos de ese bloque de código, el archivo se cierra de manera segura. Tengan en cuenta este patrón cada vez que en Python encuentren este tipo de objetos que deban ser cerrados por un método como `.close()` ya que posiblemente funcionen con `with` y de esa forma dejemos un código más claro y seguro.